In [16]:
import os
import sys
import subprocess
import numpy as np
np.set_printoptions(precision=4, suppress=True)

import cortex
from cortex.polyutils import voxelize

from scipy.ndimage.morphology import binary_erosion

from nibabel.nifti1 import Nifti1Image
from nibabel import save as nbsave, load as nbload

from matplotlib import pyplot as plt
%matplotlib inline

import seaborn as sns
import realtimefmri

# Transform white matter surface points to anatomical space

In [17]:
# subject = 'ANfs'
# xfmname = '20150722AN_auto_reading'

subject = 'SSfs_auto2'
xfmname = '20150715SS_restingstate'

In [18]:
wm_surf_pts, wm_surf_polys = cortex.db.get_surf(subject=subject,
                                                type='wm',
                                                hemisphere='both',
                                                merge=True)

anat_nifti1 = nbload('/usr/local/share/pycortex/db/'+subject+'/anatomicals/raw.nii.gz')
anat_affine = anat_nifti1.affine
wm_surf_pts_anat = np.linalg.inv(anat_affine).dot(np.r_[wm_surf_pts.T, np.ones((1,wm_surf_pts.shape[0]))]).T[:,:3]

In [19]:
wm_mask_anatref = voxelize(wm_surf_pts_anat,
                       wm_surf_polys,
                       shape=(256,256,256),
                       center=(0,0,0),
                       mp=True)

ImportError: No module named tvtk.api

In [ ]:
nbsave(Nifti1Image(wm_mask_anatref, anat_affine), '../database/'+subject+'/wm_mask_anatref.nii')

### Load white matter voxels

In [6]:
wm_mask_anatref = nbload('../database/'+subject+'/wm_mask_anatref.nii')

# White matter mask in functional reference space
anat2epispace does it all, as long as you use it correctly (transform data in and out)

In [7]:
wm_mask_funcref = cortex.volume.anat2epispace(wm_mask_anatref.get_data().T, subject, xfmname).T
wm_mask_funcref = binary_erosion(wm_mask_funcref)

In [8]:
funcref_nifti1 = cortex.db.get_xfm(subject, xfmname).reference
wm_mask_funcref = Nifti1Image(wm_mask_funcref.astype(float), funcref_nifti1.affine)

In [9]:
funcref_nifti1.affine

array([[  -2.24  ,    0.    ,    0.    ,  112.    ],
       [  -0.    ,   -2.2285,   -0.4174,  124.936 ],
       [   0.    ,   -0.2264,    4.1089,  -58.9139],
       [   0.    ,    0.    ,    0.    ,    1.    ]])

In [10]:
nbsave(funcref_nifti1, '../database/'+subject+'/funcref.nii')

In [11]:
nbsave(wm_mask_funcref, '../database/'+subject+'/wm_mask_funcref.nii')

# Visual check of white matter mask on functional reference image

In [12]:
from realtimefmri.utils import generate_command

fsl_args = [
    {
        'position': 'last',
        'value': funcref_nifti1.get_filename()
    },
    {
        'position': 'last',
        'value': '../database/'+subject+'/wm_mask_funcref.nii'
    }
]
cmd = generate_command('fslview', fsl_args)
print ' '.join(cmd)
subprocess.call(cmd)

Does fine. Now we should try morphing the gray matter mask

# Gray matter mask in functional reference space
It already is, just save voxeldata with functional reference affine in a Nifti1 file

In [12]:
gm_voxeldata = cortex.get_cortical_mask(subject, xfmname)

In [11]:
gm_voxeldata = nbload('/usr/local/share/pycortex/db/'+subject+'/transforms/'+xfmname+'/mask_thick.nii.gz')

In [ ]:
gm_voxeldata_2.T==gm_voxeldata.get_

In [14]:
gm_voxeldata.shape

(100, 100, 30)

In [15]:
gm_voxeldata_2.shape

(30, 100, 100)

In [13]:
(gm_voxeldata.get_data()==gm_voxeldata_2).all()

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  if __name__ == '__main__':


AttributeError: 'bool' object has no attribute 'all'

In [13]:
gm_mask_funcref = Nifti1Image(gm_voxeldata.astype(float), funcref_nifti1.affine)

In [18]:
nbsave(gm_mask_funcref, 'gm_mask_funcref.nii')

In [11]:
fsl_args = [
    {
        'position': 'last',
        'value': '../database/'+subject+'gm_mask_funcref.nii'
    },
    {
        'position': 'last',
        'value': '../database/'+subject+'/wm_mask_funcref.nii'
    },
    {
        'position': 'last',
        'value': '../database/'+subject+'funcref.nii'
    }
]
cmd = generate_command('fslview', fsl_args)
print ' '.join(cmd)
subprocess.call(cmd)

fslview /usr/local/share/pycortex/db/S1/transforms/20140802JG_avsnr_auto/reference.nii.gz ../database/S1/wm_mask_funcref.nii


0

# Test get mask activation
Start with an input image, 7th image in first scan

In [14]:
import realtimefmri
reload(sys.modules['realtimefmri.image_utils'])
reload(sys.modules['realtimefmri.preprocessing'])

<module 'realtimefmri.preprocessing' from '/home/glab/realtimefmri/preprocessing.pyc'>

In [15]:
r = nbload('/usr/local/share/pycortex/db/ANfs/transforms/20150722AN_auto_reading/reference.nii.gz')

In [16]:
input_affine = r.affine[:]

In [18]:
def voxel_set_explainable_variance(data, ncorrection=True, dozscore=True):
    if data.ndim == 2:
        # Fine, let's handle single voxels
        data = data[...,None]

    if dozscore:
        from scipy.stats import zscore
        data = zscore(data, 1)

    residual = data - data.mean(0)
    residualvar = np.mean(residual.var(1), 0)
    ev = 1 - residualvar

    if ncorrection:
        ev = ev - ((1 - ev) / np.float((data.shape[0] - 1)))
    return ev

In [30]:
gm_mask = cortex.db.get_mask(subject, xfmname, 'thick').astype(float)
nbsave(Nifti1Image(gm_mask.T, input_affine), '../database/'+subject+'/gm_mask_funcref.nii')

## Get activations

In [31]:
wm = realtimefmri.preprocessing.WMDetrend(subject=subject)

In [32]:
training_data = nbload('../database/'+subject+'/training_data.nii')

In [33]:
training_data.get_data().shape

(100, 100, 30, 311)

In [34]:
ntrials = training_data.shape[-1]

In [35]:
gm_activity = np.empty((ntrials, wm.masks['gm'].sum()))
wm_activity = np.empty((ntrials, wm.masks['wm'].sum()))

print gm_activity.shape
print wm_activity.shape

(19904, 80350)
(19904, 12863)


In [38]:
i = 0
print ntrials,
for i in range(ntrials):
    print i,
    volume = training_data.get_data()[...,i]
    activity = wm.get_activity_in_masks(volume)
    gm_activity[i,:] = activity['gm']
    wm_activity[i,:] = activity['wm']
    i += 1

311 0 inp_path 6f969168-0a8a-4015-9902-56579eda02e7.nii
1 inp_path bbfe4c7e-cc4c-4f98-a952-df256b527ba3.nii
2 inp_path 0ccca1eb-ba16-4f5c-84dc-2a036740580a.nii
3 inp_path f118eb76-eda3-4ed2-b432-ce3f15c985b3.nii
4 inp_path 0f9d5a26-10e9-49c9-b459-adbfe024b636.nii
5 inp_path e04e89ab-5894-4895-9a58-8e029ebff01f.nii
6 inp_path a178f97d-6b1c-46ad-b918-4711890c5fe0.nii
7 inp_path 44ce0e41-78d2-431e-9235-595939bdeac0.nii
8 inp_path a304c4cc-945b-4662-9700-65434ed4b4bf.nii
9 inp_path 330d0829-8fb2-44c9-8a0b-b34b7efe16ee.nii
10 inp_path f95235c7-d3ef-4201-8a23-5e2f2a2bdac8.nii
11 inp_path 97e4e690-9da0-4492-8887-04a127c7e7eb.nii
12 inp_path d9676d73-d282-4b45-9b40-7ade2bcf8086.nii
13 inp_path 10b08235-40cc-4412-bebc-6d182ccbc75a.nii
14 inp_path 91c8b166-f8e6-462d-b2d7-29746c22093e.nii
15 inp_path 3619f8ed-a269-4bec-95d2-0952a49c1943.nii
16 inp_path 3cc9af69-d7d5-4cc2-9058-e7cbcd7b411c.nii
17 inp_path 9432066a-6121-4ece-b90a-470bcdbe67a8.nii
18 inp_path eb0fdc38-60e4-4cf6-a9d4-7c74e2ab804a.nii

In [42]:
np.savez('../database/'+subject+'/wmd_train.npz', wm=wm_activity, gm=gm_activity)

In [ ]:
tmp = np.load('../database/'+subject+'/')

In [45]:
model, pca = wm.train(gm_activity, wm_activity)

In [84]:
import pandas as pd

In [15]:
index = pd.MultiIndex.from_tuples(zip(
        map(lambda x: os.path.split(x)[-1], scans),
        map(lambda x: os.path.split(x)[-1], timepoints)
    ), names=['scan', 'timepoint'])

gm_df = pd.DataFrame(gm_activity, index=index)
wm_df = pd.DataFrame(wm_activity, index=index)

In [16]:
gm_df.head()

0      \
scan         timepoint                                                       
pixeldata_07 20140802JG_20140802_001_007_mb1_r11a_24sl_2000m...  28.915894   
             20140802JG_20140802_001_007_mb1_r11a_24sl_2000m...  32.878357   
             20140802JG_20140802_001_007_mb1_r11a_24sl_2000m...  26.144224   
             20140802JG_20140802_001_007_mb1_r11a_24sl_2000m...  26.359467   
             20140802JG_20140802_001_007_mb1_r11a_24sl_2000m...  34.988949   

                                                                     1      \
scan         timepoint                                                       
pixeldata_07 20140802JG_20140802_001_007_mb1_r11a_24sl_2000m...  22.961060   
             20140802JG_20140802_001_007_mb1_r11a_24sl_2000m...  32.088238   
             20140802JG_20140802_001_007_mb1_r11a_24sl_2000m...  22.952639   
             20140802JG_20140802_001_007_mb1_r11a_24sl_2000m...  20.718096   
             20140802JG_20140802_001_007_mb1_r11a_24sl_2000m...  25.678940   

                                                                     2      \
scan         timepoint                                                       
pixeldata_07 20140802JG_20140802_001_007_mb1_r11a_24sl_2000m...  25.981955   
             20140802JG_20140802_001_007_mb1_r11a_24sl_2000m...  28.870239   
             20140802JG_20140802_001_007_mb1_r11a_24sl_2000m...  24.140203   
             20140802JG_20140802_001_007_mb1_r11a_24sl_2000m...  24.800652   
             20140802JG_20140802_001_007_mb1_r11a_24sl_2000m...  26.903280   

                                                                     3      \
scan         timepoint                                                       
pixeldata_07 20140802JG_20140802_001_007_mb1_r11a_24sl_2000m...  26.213217   
             20140802JG_20140802_001_007_mb1_r11a_24sl_2000m...  24.852825   
             20140802JG_20140802_001_007_mb1_r11a_24sl_2000m...  29.955080   
             20140802JG_20140802_001_007_mb1_r11a_24sl_2000m...  29.327560   
             20140802JG_20140802_001_007_mb1_r11a_24sl_2000m...  25.499571   

                                                                     4      \
scan         timepoint                                                       
pixeldata_07 20140802JG_20140802_001_007_mb1_r11a_24sl_2000m...  26.271200   
             20140802JG_20140802_001_007_mb1_r11a_24sl_2000m...  29.927734   
             20140802JG_20140802_001_007_mb1_r11a_24sl_2000m...  37.549473   
             20140802JG_20140802_001_007_mb1_r11a_24sl_2000m...  30.369879   
             20140802JG_20140802_001_007_mb1_r11a_24sl_2000m...  28.475147   

                                                                     5      \
scan         timepoint                                                       
pixeldata_07 20140802JG_20140802_001_007_mb1_r11a_24sl_2000m...  25.562927   
             20140802JG_20140802_001_007_mb1_r11a_24sl_2000m...  22.840618   
             20140802JG_20140802_001_007_mb1_r11a_24sl_2000m...  27.501450   
             20140802JG_20140802_001_007_mb1_r11a_24sl_2000m...  26.995333   
             20140802JG_20140802_001_007_mb1_r11a_24sl_2000m...  22.413933   

                                                                     6      \
scan         timepoint                                                       
pixeldata_07 20140802JG_20140802_001_007_mb1_r11a_24sl_2000m...  33.979301   
             20140802JG_20140802_001_007_mb1_r11a_24sl_2000m...  25.665466   
             20140802JG_20140802_001_007_mb1_r11a_24sl_2000m...  31.879185   
             20140802JG_20140802_001_007_mb1_r11a_24sl_2000m...  37.492733   
             20140802JG_20140802_001_007_mb1_r11a_24sl_2000m...  31.080015   

                                                                     7      \
scan         timepoint                                                       
pixeldata_07 20140802JG_20140802_001_007_mb1_r11a_24sl_2000m...  49.383175   
      

# Cross-validation
## Leave-one-out
for each dataset, predict gray matter activation using a linear regression trained on remaining datasets

In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA

In [18]:
n_wm_pcs = 10

pcas, models = [], []

gm_detrend = []
for data_dir in data_dirs:
    test_scan = os.path.split(data_dir)[-1]
    test_iloc = wm_df.index.get_level_values('scan')==test_scan
    
    wm_train = wm_df.iloc[~test_iloc].values
    gm_train = gm_df.iloc[~test_iloc].values
    
    wm_test = wm_df.iloc[test_iloc].values
    gm_test = gm_df.iloc[test_iloc].values

    n_trials, n_wm_voxels = wm_train.shape
    _, n_gm_voxels = gm_train.shape
    print data_dir,
    print 'pca',
    pca = PCA(n_components=n_wm_pcs)
    wm_train_pcs = pca.fit_transform(wm_train)
    pcas.append(pca)
    
    print 'regression',
    model = LinearRegression()
    model.fit(wm_train_pcs, gm_train)
    models.append(model)
    
    wm_test_pcs = pca.transform(wm_test)
    gm_trend = model.predict(wm_test_pcs)
    print 'done'
    gm_detrend.append(gm_test-gm_trend)

../benchmark_data/pixeldata_07 pca regression done
../benchmark_data/pixeldata_18 pca regression done
../benchmark_data/pixeldata_28 pca regression done
../benchmark_data/pixeldata_33 pca regression done
../benchmark_data/pixeldata_49 pca regression done
../benchmark_data/pixeldata_54 pca regression done
../benchmark_data/pixeldata_61 pca regression done
../benchmark_data/pixeldata_75 pca regression done
../benchmark_data/pixeldata_82 pca regression done
../benchmark_data/pixeldata_93 pca regression done


In [19]:
gm_detrend = np.asarray(gm_detrend)

save predictions

In [21]:
import cPickle

with open('pcas.pkl', 'w') as f:
    cPickle.dump(pcas, f)

with open('models.pkl', 'w') as f:
    cPickle.dump(models, f)